In [1]:
import pandas as pd
import numpy as np
import datetime
import sqlalchemy as db

s = ""
engine = db.create_engine(s)
connection = engine.connect()

In [2]:
def get_tweets_hash(project_name: str) -> pd:
    '''pulling most recent tweets'''

    project_name = project_name.replace(" ", "") 
    
    query = """ 
        SELECT text, created_at, sentiment
        FROM tweets_copy
        WHERE text_ts @@ to_tsquery('{}:*')
        AND created_at > now() - Interval '3 months'
    """.format(project_name)

    tweets = pd.read_sql_query(query, connection)
    tweets = tweets[tweets['text'].str.contains("#")].reset_index(drop=True)
    return tweets

In [3]:
### FINDING HASHTAGS FOR EACH KEYWORD
def find_tags(row_string):
    # use a list comprehension to find list items that start with #
    tags = [x for x in row_string if x.startswith('#')]
    return tags

### ask simon tomorrow -> TRUE (then includes count, if not default value)
def get_all_tags(project_name: str) -> pd:
    '''getting all relevant tags associated with an nft_project and its counts'''

    tweets = get_tweets_hash(project_name)
    hashtags_count = pd.DataFrame(tweets.text.str.extractall(r'(\#\w+)')[0].value_counts())
    hashtags_count.columns = ['count']
    hashtags_count.reset_index(inplace = True)
    hashtags_count['total'] = hashtags_count['count'].sum()
    hashtags_count['percent'] = (hashtags_count['count'] / hashtags_count['count'].sum()) * 100
    hashtags_count['timestamp'] = datetime.datetime.now()
    hashtags_count.rename(columns={ hashtags_count.columns[0]: "hashtag" }, inplace = True)
    
    #filter by count nubmer
    # result = hashtags_count[(hashtags_count['percent'] >= min_percent)]
    result = hashtags_count.head(np.minimum(20, hashtags_count.shape[0]))
    
    return result

In [9]:
get_all_tags('defi kingdom')

,hashtag,count,total,percent,timestamp
0,#Avalanche,37,521,7.101727,2022-09-08 18:31:21.224319
1,#DeFiKingdoms,37,521,7.101727,2022-09-08 18:31:21.224319
2,#GameFi,32,521,6.142035,2022-09-08 18:31:21.224319
3,#DFK,27,521,5.182342,2022-09-08 18:31:21.224319
4,#defikingdoms,18,521,3.454894,2022-09-08 18:31:21.224319
5,#AVAX,16,521,3.071017,2022-09-08 18:31:21.224319
6,#NFTCommunity,14,521,2.687140,2022-09-08 18:31:21.224319
7,#keepthisinmind,13,521,2.495202,2022-09-08 18:31:21.224319
8,#Metaverse,13,521,2.495202,2022-09-08 18:31:21.224319
9,#DeFiProject,13,521,2.495202,2022-09-08 18:31:21.224319


In [4]:
def get_all_tags_count(project_name:str) -> pd:
    '''getting all relevant tags associated with an nft_project and its counts'''
    tweets = get_tweets_hash(project_name)
    hashtags_count = pd.DataFrame(tweets.text.str.extractall(r'(\#\w+)')[0].value_counts())
    hashtags_count.columns = ['count']
    hashtags_count.reset_index(inplace = True)
    # hashtags_count['percent'] = (hashtags_count['count'] / hashtags_count['count'].sum()) * 100
    # hashtags_count['timestamp'] = datetime.datetime.now()
    # hashtags_count.rename(columns={ hashtags_count.columns[0]: "hashtag" }, inplace = True)
 
    #filter by count nubmer
    # result = hashtags_count[(hashtags_count['percent'] >= min_percent)]
    #result = hashtags_count.head(np.minimum(20, hashtags_count.shape[0]))
    return hashtags_count

In [5]:
def get_total_counts(keywords:list) -> pd:
    df0 = get_all_tags_count(keywords[0])
    df1 = get_all_tags_count(keywords[1])

    total_count = pd.merge(df0, df1, on=['index'], how='outer')
    total_count['count_x'] = total_count['count_x'].fillna(0)
    total_count['count_y'] = total_count['count_y'].fillna(0)
    total_count['count'] = total_count['count_x'] +total_count['count_y']
    total_count.drop('count_x', axis=1, inplace=True)
    total_count.drop('count_y', axis=1, inplace=True)

    # print('initialization: first total count')
    # print(total_count)

    for n in range(2,len(keywords)):
        next_df = get_all_tags_count(keywords[n])

        result = pd.merge(total_count, next_df, on=['index'], how='outer')
        result['count_x'] = result['count_x'].fillna(0)
        result['count_y'] = result['count_y'].fillna(0)
        result['count'] = result['count_x'] + result['count_y']
        result.drop('count_x', axis=1, inplace=True)
        result.drop('count_y', axis=1, inplace=True)

        # print('we are adding ' + keywords[n] + ' to the list')
        # print(next_df)
        # print(result)

        total_count = result
    
    result['normalized_prop'] = (result['count'] / result['count'].sum()) * 100
    result['total'] = result['count'].sum()
    result = result[['index', 'count', 'total', 'normalized_prop']]
    return result

In [6]:
#mytlist = ['bayc','cryptopunks','otherside','clonex', 'veefriends', 'DigiDaigaku', 'mooncats', 'CyberKongz', 'CoolCats', 'Meebits', 'MAYC', 'PudgyPenguins']

mylist = ['chikn', 'crabada', 'defi kingdom', 'heroes chained', 'imperium empires', 'open blox', 'snail trail']
result = get_total_counts(mylist)

In [16]:
result

,index,count,total,normalized_prop
0,#AVAX,153.0,2203.0,6.945075
1,#crypto,69.0,2203.0,3.132093
2,#blockchain,68.0,2203.0,3.086700
3,#trading,67.0,2203.0,3.041307
4,#Avalanche,185.0,2203.0,8.397640
...,...,...,...,...
501,#MagicStore,1.0,2203.0,0.045393
502,#newbanner,1.0,2203.0,0.045393
503,#TIMS,1.0,2203.0,0.045393
504,#GAMEFI,1.0,2203.0,0.045393


In [7]:
"""trying to get all tweets"""

defi_kingdom = get_tweets_hash('defi kingdom')
chikn = get_tweets_hash('chikn')
crabada = get_tweets_hash('crabada')
#heroes_chained = get_all_tags('HeroesChained')
#imperium_empires = get_all_tags('imperium empires')
open_blox = get_tweets_hash('open blox')
snail_trail = get_tweets_hash('snail trail')

In [8]:
totaltweets = pd.concat([defi_kingdom, chikn, crabada, open_blox, snail_trail], sort=False)

In [9]:
totaltweets

,text,created_at,sentiment
0,RT @DefiKingdoms: We wanted to share some exci...,2022-08-30 12:05:52,1.333333
1,Don't forget to cast your vote 🗳️\n\nAs of now...,2022-08-30 21:32:15,0.750000
2,Thank you @DefiKingdoms for my Birthday cake 🎂...,2022-09-06 21:24:42,2.000000
3,I really like the new look for #DFK quests 🤩\n...,2022-09-08 01:10:15,1.000000
4,🔺 @DefiKingdoms: a project combining #GameFi ...,2022-07-06 08:10:16,1.666667
...,...,...,...
68,🐌 @SnailTrailGame is now available on the @Mag...,2022-09-05 17:38:41,1.000000
69,RT @SnailTrailGame: Mega Races are now live on...,2022-08-23 04:03:57,1.000000
70,🐌 Introducing @SnailTrailGame Mega Race coming...,2022-08-19 20:04:15,1.166667
71,RT @AventuresDao: Join us for our next #GameFi...,2022-08-08 18:52:15,1.166667


In [28]:
sampling = totaltweets.sample(210)
hashtags_count = pd.DataFrame(sampling.text.str.extractall(r'(\#\w+)')[0].value_counts())
hashtags_count.columns = ['count']
hashtags_count.reset_index(inplace = True)
hashtags_count['total'] = hashtags_count['count'].sum()
hashtags_count['percent'] = (hashtags_count['count'] / hashtags_count['count'].sum()) * 100
hashtags_count['timestamp'] = datetime.datetime.now()
hashtags_count.rename(columns={ hashtags_count.columns[0]: "hashtag" }, inplace = True)

In [29]:
hashtags_count

,hashtag,count,total,percent,timestamp
0,#Avalanche,38,517,7.350097,2022-09-09 10:05:26.564255
1,#AVAX,34,517,6.576402,2022-09-09 10:05:26.564255
2,#blockchain,20,517,3.868472,2022-09-09 10:05:26.564255
3,#trading,19,517,3.675048,2022-09-09 10:05:26.564255
4,#crypto,19,517,3.675048,2022-09-09 10:05:26.564255
...,...,...,...,...,...
208,#beansong,1,517,0.193424,2022-09-09 10:05:26.564255
209,#C4B,1,517,0.193424,2022-09-09 10:05:26.564255
210,#ThePowerHour,1,517,0.193424,2022-09-09 10:05:26.564255
211,#ThePowerHouse,1,517,0.193424,2022-09-09 10:05:26.564255


In [6]:
def significant(df) :    
   return df['percent'] > df['normalized_prop']

mylist = ['chikn', 'crabada', 'defi kingdom', 'heroes chained', 'imperium empires', 'open blox', 'snail trail']
result = get_total_counts(mylist)

defi_kingdom = get_all_tags('defi kingdom')
chikn = get_all_tags('chikn')
crabada = get_all_tags('crabada')
#heroes_chained = get_all_tags('HeroesChained')
#imperium_empires = get_all_tags('imperium empires')
open_blox = get_all_tags('open blox')
snail_trail = get_all_tags('snail trail')

dupes = pd.concat([defi_kingdom, chikn, crabada, open_blox, snail_trail], sort=False).groupby('hashtag').count().sort_values(by='count', ascending=False).head(20)

In [7]:
def significant_tags(project_name:str) -> pd:   
   project_df = get_all_tags(project_name)
   project_result = project_df.merge(result.rename({'index': 'hashtag'}, axis=1), on='hashtag')
   project_result['is_significant'] = project_result.apply(significant, axis=1)
   
   project_final = project_result.merge(dupes, on='hashtag', how = 'left')
   project_final['count_y'] = project_final['count_y'].fillna(0)
   project_final = project_final.loc[(project_final['is_significant']==True)]
   project_final = project_final[['hashtag','count_x','percent_x','timestamp_x','normalized_prop','is_significant','count_y']]

   return project_final

In [155]:
significant_tags('chikn')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant,count_y
0,#AVAX,107,9.639640,2022-09-07 16:57:14.945447,7.074830,True,5.0
1,#crypto,68,6.126126,2022-09-07 16:57:14.945447,3.129252,True,0.0
2,#blockchain,68,6.126126,2022-09-07 16:57:14.945447,3.083900,True,0.0
3,#trading,67,6.036036,2022-09-07 16:57:14.945447,3.038549,True,0.0
5,#NFTs,33,2.972973,2022-09-07 16:57:14.945447,2.040816,True,3.0
6,#NFT,29,2.612613,2022-09-07 16:57:14.945447,2.267574,True,3.0
7,#chikn,29,2.612613,2022-09-07 16:57:14.945447,1.315193,True,0.0
8,#4600,28,2.522523,2022-09-07 16:57:14.945447,1.269841,True,0.0
9,#AVALANCHE,28,2.522523,2022-09-07 16:57:14.945447,1.723356,True,3.0
10,#BokBok,24,2.162162,2022-09-07 16:57:14.945447,1.088435,True,0.0


In [140]:
significant_tags('defi kingdom')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant,count_y
1,#DeFiKingdoms,36,6.779661,2022-09-06 18:14:51.965951,1.654741,True,0.0
2,#GameFi,33,6.214689,2022-09-06 18:14:51.965951,2.191413,True,2.0
3,#DFK,26,4.896422,2022-09-06 18:14:51.965951,1.207513,True,0.0
4,#defikingdoms,18,3.389831,2022-09-06 18:14:51.965951,0.805009,True,0.0
6,#NFTCommunity,14,2.636535,2022-09-06 18:14:51.965951,1.118068,True,2.0
7,#Crystalvale,13,2.448211,2022-09-06 18:14:51.965951,0.581395,True,0.0
8,#DeFiProject,13,2.448211,2022-09-06 18:14:51.965951,0.581395,True,0.0
9,#CryptoCommunity,13,2.448211,2022-09-06 18:14:51.965951,0.581395,True,0.0
10,#Metaverse,13,2.448211,2022-09-06 18:14:51.965951,0.581395,True,0.0
11,#keepthisinmind,13,2.448211,2022-09-06 18:14:51.965951,0.581395,True,0.0


In [8]:
def unique_to_proj_tags(project_name:str) -> pd:   
   project_df = get_all_tags(project_name)
   project_result = project_df.merge(result.rename({'index': 'hashtag'}, axis=1), on='hashtag')
   project_result['is_significant'] = project_result.apply(significant, axis=1)
   
   project_final = project_result.merge(dupes, on='hashtag', how = 'left')
   project_final['count_y'] = project_final['count_y'].fillna(0)
   project_final = project_final.loc[(project_final['is_significant']==True) & (project_final['count_y']==0)]
   project_final = project_final[['hashtag','count_x','percent_x','timestamp_x','normalized_prop','is_significant']]
   # project_final = project_final.loc[~project_final['hashtag'].str.contains('crypto', case=False)]
   # project_final = project_final.loc[~project_final['hashtag'].str.contains('defiproject', case=False)]
   # project_final = project_final.loc[~project_final['hashtag'].str.contains('metaverse', case=False)]

   return project_final

In [11]:
unique_to_proj_tags('defi kingdom')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant
1,#DeFiKingdoms,37,7.061069,2022-09-08 09:13:58.941232,1.721794,True
3,#DFK,27,5.152672,2022-09-08 09:13:58.941232,1.268691,True
4,#defikingdoms,18,3.435115,2022-09-08 09:13:58.941232,0.815587,True
7,#Metaverse,13,2.480916,2022-09-08 09:13:58.941232,0.589035,True
8,#CryptoCommunity,13,2.480916,2022-09-08 09:13:58.941232,0.589035,True
9,#keepthisinmind,13,2.480916,2022-09-08 09:13:58.941232,0.589035,True
10,#DeFiProject,13,2.480916,2022-09-08 09:13:58.941232,0.589035,True
11,#Crystalvale,13,2.480916,2022-09-08 09:13:58.941232,0.589035,True
12,#cryptocurrencies,12,2.290076,2022-09-08 09:13:58.941232,0.589035,True
13,#cryptocurrency,11,2.099237,2022-09-08 09:13:58.941232,0.498414,True


In [143]:
unique_to_proj_tags('chikn')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant
1,#crypto,69,6.106195,2022-09-06 18:15:18.428435,3.130590,True
2,#blockchain,69,6.106195,2022-09-06 18:15:18.428435,3.085868,True
3,#trading,68,6.017699,2022-09-06 18:15:18.428435,3.041145,True
7,#chikn,29,2.566372,2022-09-06 18:15:18.428435,1.296959,True
9,#4600,28,2.477876,2022-09-06 18:15:18.428435,1.252236,True
10,#BokBok,24,2.123894,2022-09-06 18:15:18.428435,1.073345,True
11,#AVAXPDX,18,1.592920,2022-09-06 18:15:18.428435,0.805009,True
12,#4400,17,1.504425,2022-09-06 18:15:18.428435,0.760286,True
13,#bokbok,17,1.504425,2022-09-06 18:15:18.428435,0.760286,True
17,#ChiknAndHoops,13,1.150442,2022-09-06 18:15:18.428435,0.581395,True


In [144]:
unique_to_proj_tags('crabada')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant
1,#snibsnib,18,7.438017,2022-09-06 18:15:27.835978,0.849732,True
3,#CrabadaLore,9,3.719008,2022-09-06 18:15:27.835978,0.402504,True
4,#SnakeCity,7,2.892562,2022-09-06 18:15:27.835978,0.313059,True
7,#crabada,5,2.066116,2022-09-06 18:15:27.835978,0.223614,True
9,#CRUSTACOTIMES,5,2.066116,2022-09-06 18:15:27.835978,0.223614,True
11,#gamefi,4,1.652893,2022-09-06 18:15:27.835978,0.178891,True
12,#1,4,1.652893,2022-09-06 18:15:27.835978,0.402504,True
13,#Chainlink,4,1.652893,2022-09-06 18:15:27.835978,0.178891,True
14,#Sn,3,1.239669,2022-09-06 18:15:27.835978,0.134168,True
15,#avalanche,3,1.239669,2022-09-06 18:15:27.835978,0.357782,True


In [145]:
unique_to_proj_tags('open blox')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant
1,#OpenBlox,15,7.352941,2022-09-06 18:15:47.491033,0.670841,True
2,#Web3,15,7.352941,2022-09-06 18:15:47.491033,0.849732,True
5,#DeFi,9,4.411765,2022-09-06 18:15:47.491033,0.626118,True
10,#Bybit,4,1.960784,2022-09-06 18:15:47.491033,0.178891,True
15,#Launchpad,3,1.470588,2022-09-06 18:15:47.491033,0.134168,True
16,#BloxVerse,2,0.980392,2022-09-06 18:15:47.491033,0.089445,True
17,#Blox,2,0.980392,2022-09-06 18:15:47.491033,0.089445,True
19,#SOL3S,2,0.980392,2022-09-06 18:15:47.491033,0.089445,True


In [146]:
unique_to_proj_tags('snail trail')

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_significant
3,#SlimeGang,6,4.651163,2022-09-06 18:16:23.433659,0.268336,True
8,#SnailTrail,3,2.325581,2022-09-06 18:16:23.433659,0.178891,True
9,#AvaxWeekly,2,1.550388,2022-09-06 18:16:23.433659,0.089445,True
10,#Games,2,1.550388,2022-09-06 18:16:23.433659,0.089445,True
11,#Dexalot,2,1.550388,2022-09-06 18:16:23.433659,0.089445,True
12,#NFTNYC,2,1.550388,2022-09-06 18:16:23.433659,0.089445,True
13,#GAMEFI,1,0.775194,2022-09-06 18:16:23.433659,0.044723,True
16,#AvaxNFTs,1,0.775194,2022-09-06 18:16:23.433659,0.134168,True
17,#TaleCraft,1,0.775194,2022-09-06 18:16:23.433659,0.089445,True
18,#Avaxtars,1,0.775194,2022-09-06 18:16:23.433659,0.089445,True


In [113]:
# # gamefi_df = get_all_tags('gamefi',20)
# # gamefi_df.to_csv('gamefi.csv')
# # df = pd.read_csv('gamefi.csv')
# defi_kingdom = get_all_tags('defi kingdom')
# chikn = get_all_tags('chikn')
# crabada = get_all_tags('crabada')
# #heroes_chained = get_all_tags('HeroesChained')
# #imperium_empires = get_all_tags('imperium empires')
# open_blox = get_all_tags('open blox')
# snail_trail = get_all_tags('snail trail')

# mylist = ['chikn', 'crabada', 'defi kingdom', 'heroes chained', 'imperium empires', 'open blox', 'snail trail']
# result = get_total_counts(mylist)

# defi_kingdom_result = defi_kingdom.merge(result.rename({'index': 'hashtag'}, axis=1), on='hashtag')
# def significant(df) :    
#    return df['percent'] > df['normalized_prop']

# defi_kingdom_result['is_it_significant'] = defi_kingdom_result.apply(significant, axis=1)

# dupes = pd.concat([defi_kingdom, chikn, crabada, open_blox, snail_trail], sort=False).groupby('hashtag').count().sort_values(by='count', ascending=False).head(20)

# defi_kingdom_final = defi_kingdom_result.merge(dupes, on='hashtag', how = 'left')
# defi_kingdom_final['count_y'] = defi_kingdom_final['count_y'].fillna(0)
# defi_kingdom_final = defi_kingdom_final.loc[(defi_kingdom_final['is_it_significant']==True) & (defi_kingdom_final['count_y']==0)]
# defi_kingdom_final = defi_kingdom_final[['hashtag','count_x','percent_x','timestamp_x','normalized_prop','is_it_significant']]
# # defi_kingdom_final = defi_kingdom_final.loc[~defi_kingdom_final['hashtag'].str.contains('crypto', case=False)]
# # defi_kingdom_final = defi_kingdom_final.loc[~defi_kingdom_final['hashtag'].str.contains('defiproject', case=False)]
# defi_kingdom_final

,hashtag,count_x,percent_x,timestamp_x,normalized_prop,is_it_significant
1,#DeFiKingdoms,36,6.779661,2022-09-06 18:00:04.433521,1.654741,True
3,#DFK,26,4.896422,2022-09-06 18:00:04.433521,1.207513,True
4,#defikingdoms,18,3.389831,2022-09-06 18:00:04.433521,0.805009,True
7,#Crystalvale,13,2.448211,2022-09-06 18:00:04.433521,0.581395,True
8,#DeFiProject,13,2.448211,2022-09-06 18:00:04.433521,0.581395,True
9,#CryptoCommunity,13,2.448211,2022-09-06 18:00:04.433521,0.581395,True
10,#Metaverse,13,2.448211,2022-09-06 18:00:04.433521,0.581395,True
11,#keepthisinmind,13,2.448211,2022-09-06 18:00:04.433521,0.581395,True
12,#cryptocurrencies,12,2.259887,2022-09-06 18:00:04.433521,0.581395,True
14,#cryptocurrency,11,2.071563,2022-09-06 18:00:04.433521,0.491950,True


In [ ]:
### GENERATING WORDCLOUD FOR EACH KEYWORD
# def get_wordcloud(nft_project):
#     count = 30
#     result = get_all_tags(nft_project, count)
#     data = result.set_index('index').to_dict()['count']

#     #generating wordcloud
#     wc = WordCloud(width=800, height=400, random_state=1, background_color='white', colormap='Set2', collocations=False).generate_from_frequencies(data)

#     plt.imshow(wc)
#     plt.axis('off')
#     plt.show()  

In [ ]:
# # convert to dict
# data = result1.set_index('index').to_dict()['count']

# # wc = WordCloud(width=800, height=400, max_words=200).generate_from_frequencies(data)
# wc = WordCloud(width=800, height=400, random_state=1, background_color='white', colormap='Set2', collocations=False).generate_from_frequencies(data)

# # #presentation with mask
# # mask = np.array(Image.open("../input/input-img/Twitter.png"))
# # wc = WordCloud(width=800, height=400, random_state=1, background_color='black', colormap='Set2', collocations=False, mask=mask).generate_from_frequencies(data)

# plt.imshow(wc)
# plt.axis('off')
# plt.show()

In [ ]:
# relevant_twts = get_tweets_text('nft')
# relevant_twts
# # wordcounts_from_relev = relevant_twts.processed_text.str.split(expand=True).stack().value_counts()
# # wordcounts_from_relev.name = 'counts'
# # wordcounts_from_relev = pd.DataFrame(wordcounts_from_relev)
# # wordcounts_from_relev['prop'] = wordcounts_from_relev['counts']/wordcounts_from_relev['counts'].sum()
# # wordcounts_from_relev.reset_index().rename(columns={'index': 'keyword'})
# # wordcounts_from_relev.head(10)

In [ ]:
# for index, row in relevant_twts.iterrows():
#     relevant_twts['splitted_texts'] = relevant_twts['text'].str.split()
# relevant_twts['tags'] = relevant_twts['splitted_texts'].apply(lambda row : find_tags(row))

### replace # as requested in OP, replace for new lines and \ as needed.
# # df['tags'] = df['tags'].apply(lambda x : str(x).replace('#', '').replace('\\n', ',').replace('\\', '').replace("'", ""))